
<!-- <head>
    <link rel="stylesheet" href="{{site.baseurl}}/assets/Bathroom/bathroom.css">
</head> -->

{% include nav/toolkits/bathroom/menu.html %}


<div class="container">
    <div class="components">
        <!-- queue header and your info like eta -->
        <div class="div1">
            <div class="queue-header">
                <h2 id="teacherName">Mr. Mortensen's Queue</h2>
            </div>

            <div class="queue-info">
                <p id="position">Your position in line: #</p>
                <p id="estimatedTime">Estimated time: ~ mins</p>
            </div>

            <hr>
        </div>

        <!-- students in line -->
        <div class="div2">
            <div class="queue-list">
                <p>Students currently in line:</p>
                <ul id="studentList">
                    <!-- Students will be listed here -->
                </ul>

                <div class="queue-buttons queue-actions">
                    <button type="button" onclick="window.addToQueue()">Add Me to Queue</button>
                    <button type="button" onclick="window.removeFromQueue()">Remove Me from Queue</button>
                    <button type="button" href="{{site.baseurl}}/bathroom/issues">Issues</button>
                </div>
            </div>
        </div>
    </div>
</div>

<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script type="module" src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.esm.js"></script>
<script nomodule src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.js"></script>

<script type="module">
    import {javaURI} from '{{site.baseurl}}/assets/js/api/config.js';

    // Global variable to store queue data
    window.studentsInQueue = [];

    window.displayQueue = function() {
        const studentList = document.getElementById("studentList");
        studentList.innerHTML = ""; // Clear the list

        // Display the queue
        window.studentsInQueue.forEach((student) => {
            const listItem = document.createElement("li");
            listItem.textContent = student;
            if (student === studentName) listItem.style.color = "red"; // Highlight user's name in red
            studentList.appendChild(listItem);
        });

        // Disable "Add Me to Queue" button if student is already in queue
        const addButton = document.querySelector('button[onclick="window.addToQueue()"]');
        if (window.studentsInQueue.includes(studentName)) {
            addButton.disabled = true;
        } else {
            addButton.disabled = false;
        }

        // Update position and estimated time
        const position = window.studentsInQueue.indexOf(studentName) + 1;
        document.getElementById("position").textContent = `Your position in line: #${position}`;
        document.getElementById("estimatedTime").textContent = `Estimated time: ~${position * 3} mins`;
    }

    // Initial display
    window.displayQueue();
</script>

<script type="module">
    import {javaURI} from '{{site.baseurl}}/assets/js/api/config.js';

    const url = `${javaURI}/api/queue`;
    const getUrl = url + '/all';
    const addUrl = url + '/add';
    const removeUrl = url + '/remove';
    const approveUrl = url + '/approve';

    const fetchOptions = {
        headers: {
            "Content-Type": "application/json"
        }
    };

    const postOptions = {
        ...fetchOptions,
        method: 'POST',
    };
    const deleteOptions = {
        ...fetchOptions,
        method: 'DELETE',
    };

    // Get teacher name and student name from local storage or variables
    const teacherName = "jm1021@gmail.com"; // Replace with dynamic teacher name if needed
    const studentName = localStorage.getItem("name"); // Assume stored from login

    window.fetchQueueData = function() {
        fetch(getUrl, fetchOptions)
            .then(response => {
                if (response.status !== 200) {
                    console.error("Failed to fetch queue data.");
                    return;
                }
                return response.json();
            })
            .then(data => {
                console.log("Queue data:", data);
                const mortensenQueue = data.find(queue => queue.teacherEmail === teacherName);
                if (mortensenQueue) {
                    window.studentsInQueue = mortensenQueue.peopleQueue.split(','); // Convert the queue string into an array
                    window.displayQueue();

                    // Run approveStudent if there is at least one student in queue and the first student is the current user
                    if (window.studentsInQueue.length > 0 && window.studentsInQueue[0] === studentName) {
                        window.approveStudent();
                    }

                    if (mortensenQueue.away === 1 && window.studentsInQueue.length > 0) {
                        window.studentsInQueue[0] += " (in bathroom)";
                        window.displayQueue();
                    }
                } else {
                    console.error("Mortensen's queue not found.");
                }
            })
            .catch(error => console.error("Error fetching data:", error));
    }

    // Function to add student to the queue
    window.addToQueue = function() {
        const queuer = {
            teacherEmail: teacherName,
            studentName: studentName,
        };

        fetch(addUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) {
                    window.fetchQueueData(); // Refresh the queue after adding
                } else {
                    alert("Failed to add to queue.");
                }
            })
            .catch(error => console.error("Error adding to queue:", error));
        console.log("Added to queue:", queuer);
    }

    // Function to remove student from the queue
    window.removeFromQueue = function() {
        const queuer = {
            teacherEmail: teacherName,
            studentName: studentName,
        };

        fetch(removeUrl, {
            ...deleteOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) {
                    window.fetchQueueData(); // Refresh the queue after removing
                } else {
                    alert("Failed to remove from queue.");
                }
            })
            .catch(error => console.error("Error removing from queue:", error));
        console.log("Removed from queue:", queuer);
    }

    // Function to approve student who is first in line
    window.approveStudent = function() {
        const now = new Date();
        const hours = now.getHours();
        const minutes = now.getMinutes().toString().padStart(2, "0"); // Ensures 2 digits
        const seconds = now.getSeconds().toString().padStart(2, "0");
        const timeIn = `${hours}:${minutes}:${seconds}`;
        localStorage.setItem("timeIn", timeIn);

        const queuer = {
            teacherEmail: teacherName,
            studentName: window.studentsInQueue[0],
        };

        fetch(approveUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) {
                    window.location.href = "{{site.baseurl}}/hallpass";
                }
                else {
                    alert("Failed to approve student.");
                }
            })
            .catch(error => console.error("Error approving student:", error));
    }

    // Fetch data every 10 seconds
    setInterval(window.fetchQueueData, 10000);

    // Initial fetch
    window.fetchQueueData();
</script>